In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("D:\Backup\Desktop\Projects\Feep_Learning_Playlist\gender_classification_v7.csv")
df.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


In [3]:
gender = pd.get_dummies(df['gender'],drop_first=True)

In [4]:
df['gender'].value_counts()

Female    2501
Male      2500
Name: gender, dtype: int64

In [5]:
df = pd.concat([df,gender],axis = 1)

In [6]:
df.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,Male
0,1,11.8,6.1,1,0,1,1,Male,1
1,0,14.0,5.4,0,0,1,0,Female,0
2,0,11.8,6.3,1,1,1,1,Male,1
3,0,14.4,6.1,0,1,1,1,Male,1
4,1,13.5,5.9,0,0,0,0,Female,0


In [7]:
df.drop(['gender'],axis=1,inplace=True)

In [8]:
df.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,Male
0,1,11.8,6.1,1,0,1,1,1
1,0,14.0,5.4,0,0,1,0,0
2,0,11.8,6.3,1,1,1,1,1
3,0,14.4,6.1,0,1,1,1,1
4,1,13.5,5.9,0,0,0,0,0


In [9]:
x = df.iloc[:,:7]
y = df.iloc[:,7:]

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [11]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [12]:
import keras_tuner
from keras_tuner import RandomSearch

In [13]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layer',2,15)):
        model.add(layers.Dense(units = hp.Int('units_'+str(i),
                                             min_value = 32,
                                             max_value=512,
                                             step = 32),
                              activation = hp.Choice('activation_' + str(i), values=['sigmoid','relu','tanh'])))
        model.add(layers.Dense(1,activation = 'sigmoid'))
        model.compile(optimizer = keras.optimizers.Adam(
        hp.Choice('learning_rate',[1e-3,1e-4,1e-5])),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])
    return model

In [15]:
tuner = RandomSearch(
build_model,
objective = 'val_accuracy',
max_trials = 4,
executions_per_trial=3,
directory='project3',
project_name='Gender Classification')

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
num_layer (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 15, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation_0 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation_1 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}


In [17]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 4 Complete [00h 00m 33s]
val_accuracy: 0.4970019857088725

Best val_accuracy So Far: 0.9562513828277588
Total elapsed time: 00h 02m 25s


In [18]:
tuner.results_summary()

Results summary
Results in project3\Gender Classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layer: 2
units_0: 448
activation_0: relu
learning_rate: 0.0001
units_1: 480
activation_1: tanh
units_2: 160
activation_2: relu
units_3: 352
activation_3: sigmoid
units_4: 256
activation_4: relu
units_5: 320
activation_5: tanh
units_6: 352
activation_6: relu
units_7: 384
activation_7: relu
units_8: 192
activation_8: relu
Score: 0.9562513828277588

Trial 0 summary
Hyperparameters:
num_layer: 9
units_0: 512
activation_0: tanh
learning_rate: 0.0001
units_1: 384
activation_1: tanh
units_2: 32
activation_2: sigmoid
units_3: 32
activation_3: sigmoid
units_4: 32
activation_4: sigmoid
units_5: 32
activation_5: sigmoid
units_6: 32
activation_6: sigmoid
units_7: 32
activation_7: sigmoid
units_8: 32
activation_8: sigmoid
Score: 0.4970019857088725

Trial 1 summary
Hyperparameters:
num_layer: 5
units_0: 64
activation_0: tanh
learning_r

In [19]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5121 - loss: 0.6917
Epoch 2/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4933 - loss: 0.6718
Epoch 3/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7234 - loss: 0.5949
Epoch 4/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9537 - loss: 0.4693
Epoch 5/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9502 - loss: 0.3483


In [20]:
y_pred = model.predict(x_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [24]:
y_pred_labels = np.where(y_pred >= 0.5, 1, 0)
y_pred_labels

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)
roc_auc = roc_auc_score(y_test, y_pred)  # Note: Use probabilities for ROC-AUC

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Accuracy: 0.9560293137908061
Precision: 0.9295774647887324
Recall: 0.9850746268656716
F1 Score: 0.9565217391304348
ROC-AUC Score: 0.9916058096002613
